## The purpose of the Capstone 2 project is to determine whether or not the lesion in an image is malignant or not 

### This script loops over the traditional image processing method to produce statistics on malignant and benign lesions ###

Input: image(.jpg)<br>
Output: dictionary (imgname:original img, segmented image,largest contour,area of the largest contour,mask used for segmentation)

In [2]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

In [1]:
%run functions.py

Using TensorFlow backend.
/home/seo/environments/my_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/seo/environments/my_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/seo/environments/my_env/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/seo/environments/my_env/lib/p

In [2]:
def split_into_rgb_channels(image):
    """Split the target image into its red, green and blue channels.image - a numpy array of shape (rows, columns, 3).
    output - three numpy arrays of shape (rows, columns) and dtype same as
    image, containing the corresponding channels.
    """
    red = image[:,:,2]
    green = image[:,:,1]
    blue = image[:,:,0]
    return red, green, blue

In [3]:
def rotate_bound(image, angle,cX,cY):
    # grab the dimensions of the image
    (h, w) = image.shape[:2] 
    # grab the rotation matrix, then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
 
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
 
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
 
    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))


In [4]:
def mse(imageA, imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(maxContour)
    # return the MSE, the lower the error, the more "similar"
    # the two images are
    return err

In [5]:
def get_mag_ang(img):

    """
    Gets image gradient (magnitude) and orientation (angle)

    Args:
        img

    Returns:
        Gradient, orientation
    """

    img = np.sqrt(img)

    gx = cv2.Sobel(np.float32(img), cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(np.float32(img), cv2.CV_32F, 0, 1)

    mag, ang = cv2.cartToPolar(gx, gy)

    return mag, ang, gx, gy 

In [3]:
symmetry_dict=dict()
blue_dict=dict()
green_dict=dict()
red_dict=dict()
border_dict=dict()

In [4]:
path="/home/seo/ISIC_DATA"

import pickle
name="isic_data"
df=pickle.load(open(name,"rb"))

import cv2

filelist=os.listdir(path)
pics=[file for file in filelist if (file.endswith(".jpeg") or file.endswith(".png")) and file.startswith("ISIC")]


In [ ]:
j=0
for i in range(j,len(pics)):

    imgname=pics[i]
    filename=path+'/'+pics[i]
    
    label=imgname.split('.')[0]

    orig_img=cv2.imread(filename, cv2.IMREAD_COLOR)
    img=orig_img

    # normalize image

    norm_image = cv2.normalize(img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)

    img=norm_image

    # split image into RGB channels



    r,g,b=split_into_rgb_channels(img)

    
    #reduce noise

    # Do some denoising on blue channel because that usually gives best contrast
    gaussian = cv2.GaussianBlur(b,(3,3),0)


    #find edge & image segmentation

    kernel = np.ones((5,5),np.uint8)
    th, threshed = cv2.threshold(gaussian, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    erosion = cv2.erode(threshed,kernel,iterations = 3)
    dilation = cv2.morphologyEx(erosion, cv2.MORPH_OPEN, kernel)

    cnts, hierarchy=cv2.findContours(erosion,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE )
    cnts = sorted(cnts, key=cv2.contourArea)
    H,W = img.shape[:2]
    
    maxContour = 0

    for contour in cnts:
        cv2.drawContours(img,contour,-1,(255,0,0),-1)
        contourSize = cv2.contourArea(contour)
        if contourSize > maxContour:
            maxContour = contourSize
            maxContourData = contour
        #if contour
    cv2.drawContours(img,[contour],-1,(255,0,0),-1)

    # Create mask and do bitwise-op
    
    mask = np.zeros(img.shape[:2],np.uint8)
    cv2.drawContours(mask, [contour],-1, 255, -1)
    
    #area = cv2.contourArea(contour)
    dst = cv2.bitwise_and(gaussian, gaussian, mask=mask)
    
    fourcorners=[(0,0),(img.shape[:2][1]-1,0),(0,img.shape[:2][0]-1),(img.shape[:2][1]-1,img.shape[:2][0]-1)]

    orig_cnts=cnts
    new_cnts=list()
    for j in range(len(cnts)):
        try:
            contour=cnts[j]
            bunch=[tuple(contour[num][0]) for num in range(len(contour))]
            y_min=min(bunch, key=lambda t: t[0])[0]
            y_max=max(bunch, key=lambda t: t[0])[0]
            x_min=min(bunch, key=lambda t: t[1])[1]
            x_max=max(bunch, key=lambda t: t[1])[1]

            y_count=0
            x_count=0

            for num in bunch:
                if num[0]==x_min:
                    x_count=x_count+1
                if num[0]==x_max:
                    x_count=x_count+1
                if num[1]==y_min:
                    y_count=y_count+1
                if num[1]==y_max:
                    y_count=y_count+1
            if (x_count>round(img.shape[:2][0]*0.6)) or (y_count>round(img.shape[:2][1]*0.6)):
                cnts.pop(j)
        except:
            continue
    #retread to check if there are removed contour
    for contour in cnts:
        cv2.drawContours(img,contour,-1,(255,0,0),-1)
        contourSize = cv2.contourArea(contour)
        if contourSize > maxContour:
            maxContour = contourSize
            maxContourData = contour
        #if contour
    cv2.drawContours(img,[contour],-1,(255,0,0),-1)
    ## Create mask and do bitwise-op
    mask = np.zeros(img.shape[:2],np.uint8)
    cv2.drawContours(mask, [contour],-1, 255, -1)
    #area = cv2.contourArea(contour)
    dst = cv2.bitwise_and(gaussian, gaussian, mask=mask)

   
    #Check for symmetry: bigger value more malignant

    # get angle and center for rotation
    (a,b),(MA,ma),angle = cv2.fitEllipse(maxContourData)
    a=int(a)
    b=int(b)


    rotated_roi=rotate_bound(dst,angle,a,b)




    from mpl_toolkits.axes_grid1 import AxesGrid

    imageO=rotated_roi
    imageLR=np.fliplr(rotated_roi)
    imageUD=np.flipud(rotated_roi)

    lr=mse(rotated_roi, np.fliplr(rotated_roi),maxContour)
    ud=mse(rotated_roi, np.flipud(rotated_roi),maxContour)

    symmetry_dict[label]=(lr+ud)/2

    #Check Color: bigger value more malignant

    colorcheck = cv2.bitwise_and(orig_img, orig_img, mask=mask)
    #plt.imshow(colorcheck)

    color = ('b','g','r')
    #fig=plt.figure()
    zerovalues=list()
    #ax=fig.add_subplot(2,1,1)
    b_list=[]
    g_list=[]
    r_list=[]
    for i,col in enumerate(color):
        histr = cv2.calcHist([colorcheck],[i],None,[256],[0,256])
        #ax.plot(histr,color = col)
        zerovalues.append(histr[0])
        histr=histr[1:]
        #plt.xlim([0,255])
        #ax.set_yscale('log')
        if col=='b':
            blue_dict[label]=histr.std()
        if col=='g':
            green_dict[label]=histr.std()
        if col=='r':
            red_dict[label]=histr.std()
        #print(histr.std())
    #plt.show()
    #zerovalues
    #print(type(std_list))
    #print(std_list)

    #Check border



    h=20; w=20
    tots_gradient=[]
    for point in maxContourData[:,0]:
        x=point[0]
        y=point[1]
        #print(x,y)
        y1=y-h; y2=y+h
        x1=x-w; x2=x+w
        if y1<0:
            y1=0
        if y2>H:
            y2=H
        if x1<0:
            x1=0
        if x2>W:
            x2=W
        #print(x1,y1,x2,y2)
        crop_img = orig_img[:,:,0][y1:y2, x1:x2].copy()
        mag,ang,gx,gy=get_mag_ang(crop_img)
        tots_gradient.append(mag.mean())

    border_dict[label]=np.mean(tots_gradient)
    j=j+1


print(symmetry_dict,blue_dict,border_dict)

In [9]:
with open('sym_data.pickle', 'wb') as f:
    pickle.dump(symmetry_dict, f)
with open('b_data.pickle', 'wb') as f:
    pickle.dump(blue_dict, f)
with open('g_data.pickle', 'wb') as f:
    pickle.dump(green_dict, f)
with open('r_data.pickle', 'wb') as f:
    pickle.dump(red_dict, f)
with open('bor_data.pickle', 'wb') as f:
    pickle.dump(border_dict, f)